# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
res = 0
for sqrt_value in generator:
    print(sqrt_value)
    res+=sqrt_value

res


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


8.382332347441762

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
import dlt 
import duckdb

pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# data_1=[]
# for person in people_1():
#     print(person)
#     data_1.append(person)

info = pipeline.run(people_1(),
					table_name="people",
					write_disposition="append",
                    primary_key='ID')

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

conn.sql('SELECT * FROM people')
 

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708074720.0990245 │ 3zgx71+pe4cCuQ │
│     2 │ Person_2 │    27 │ City_A  │ 1708074720.0990245 │ Vj+XCbV88q9yiw │
│     3 │ Person_3 │    28 │ City_A  │ 1708074720.0990245 │ pMrtjjSgSgC47A │
│     4 │ Person_4 │    29 │ City_A  │ 1708074720.0990245 │ fhQt9oACdBySkw │
│     5 │ Person_5 │    30 │ City_A  │ 1708074720.0990245 │ L5i3YkRL6DgD/Q │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘

In [3]:
# calculate the sum of ages of all people from the first generator
conn.sql('SELECT SUM(Age) FROM people')

┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [4]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# data_2=[]
# for person in people_2():
#     print(person)
#     data_2.append(person)

info = pipeline.run(people_2,
					table_name="people",
					write_disposition="append",
                    primary_key='ID')

In [10]:
conn.sql('SELECT * FROM people')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708074720.0990245 │ 3zgx71+pe4cCuQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708074720.0990245 │ Vj+XCbV88q9yiw │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708074720.0990245 │ pMrtjjSgSgC47A │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708074720.0990245 │ fhQt9oACdBySkw │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708074720.0990245 │ L5i3YkRL6DgD/Q │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708074721.2445679 │ d6YIZiyunRJdlg │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708074721.2445679 │ 8nsN3Gj0eP6/zw │ Job_4      │
│     5 │ 

In [5]:
# calculate the sum of ages of all people 
conn.sql('SELECT SUM(Age) FROM people')

┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [6]:
#Install the dependencies
# %%capture
# !pip install dlt[duckdb]

In [7]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

info = pipeline.run(people_1(),table_name='people_merged',write_disposition='append',primary_key='ID')
info = pipeline.run(people_2(),table_name='people_merged',write_disposition='merge',primary_key='ID')

In [9]:
conn.sql('SELECT * FROM people.people_merged')

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708074722.0906181 │ JCkZprQqVdKaRg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708074722.0906181 │ 540vFrebrRaL2g │ NULL       │
│     7 │ Person_7 │    37 │ City_B  │ 1708074722.8752172 │ mBv1Z6q1tSZtWw │ Job_7      │
│     8 │ Person_8 │    38 │ City_B  │ 1708074722.8752172 │ 9hgtBQNxxiWldA │ Job_8      │
│     5 │ Person_5 │    35 │ City_B  │ 1708074722.8752172 │ F4i6gR/wlA5GVQ │ Job_5      │
│     4 │ Person_4 │    34 │ City_B  │ 1708074722.8752172 │ ySQYSq9xYgETHQ │ Job_4      │
│     3 │ Person_3 │    33 │ City_B  │ 1708074722.8752172 │ 1jtZPF8IP5CMyw │ Job_3      │
│     6 │ 

In [8]:
conn.sql('SELECT SUM(Age) FROM people.people_merged')

┌──────────┐
│ sum(Age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX